In [15]:
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from sklearn.metrics import roc_curve, auc, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import mixed_precision
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from sklearn.metrics import roc_curve, auc, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import EfficientNetB0,MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import mixed_precision
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard

In [4]:
img_size = (224, 224)
batch_size = 18

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],
    channel_shift_range=20,
)

train_dir = "/kaggle/input/feb-25-702010-trvaltest/3.tr70val20te10/train"
val_dir = "/kaggle/input/feb-25-702010-trvaltest/3.tr70val20te10/val"
test_dir = "/kaggle/input/feb-25-702010-trvaltest/3.tr70val20te10/test"

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 20827 images belonging to 2 classes.
Found 5952 images belonging to 2 classes.
Found 2975 images belonging to 2 classes.


In [33]:
os.remove('/kaggle/working/model_weights.h5')

In [34]:
from tensorflow.keras.applications.resnet50 import ResNet50

base_model = ResNet50(input_shape = (224, 224, 3),include_top = False,weights = "imagenet")
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [35]:
for layer in base_model.layers:
    layer.trainable = False

In [36]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
callbacks1 = [
    ModelCheckpoint(filepath='model_weights.h5', save_best_only=True),
    EarlyStopping(patience=5,monitor='val_loss', restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001, verbose=1)]

In [38]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=callbacks1
)

Epoch 1/20
1158/1158 [==============================] - 339s 290ms/step - loss: 0.6291 - accuracy: 0.6225 - val_loss: 0.6295 - val_accuracy: 0.6154 - lr: 0.0010
Epoch 2/20
1158/1158 [==============================] - 313s 270ms/step - loss: 0.5597 - accuracy: 0.6990 - val_loss: 0.5256 - val_accuracy: 0.6983 - lr: 0.0010
Epoch 3/20
1158/1158 [==============================] - 316s 273ms/step - loss: 0.5313 - accuracy: 0.7242 - val_loss: 0.4978 - val_accuracy: 0.7930 - lr: 0.0010
Epoch 4/20
1158/1158 [==============================] - 317s 274ms/step - loss: 0.5150 - accuracy: 0.7387 - val_loss: 0.5701 - val_accuracy: 0.6270 - lr: 0.0010
Epoch 5/20
1158/1158 [==============================] - 320s 276ms/step - loss: 0.5006 - accuracy: 0.7506 - val_loss: 0.4441 - val_accuracy: 0.7823 - lr: 0.0010
Epoch 6/20
1158/1158 [==============================] - 320s 276ms/step - loss: 0.4900 - accuracy: 0.7585 - val_loss: 0.4343 - val_accuracy: 0.8258 - lr: 0.0010
Epoch 7/20
1158/1158 [============

In [39]:
model.save('ResNet50_Feb_25_702010_imagenetpretrainmodel.h5')